In [ ]:
import os
# 在继续之前，请确保已经设置了用于调用大模型的 API Key。
# 一般来说，只需设置 application.yaml 中 llm.qwen.api_key_env 指向的环境变量即可。
#
# 例如：
#   在 Windows:
#       set API_KEY_QWEN=你的密钥
#   在 macOS / Linux:
#       export API_KEY_QWEN=你的密钥
#
# 如未设置，将无法构建向量库（embedding 会依赖该 Key）。

env_name = "API_KEY_QWEN"
if not os.environ.get(env_name):
    print(f"注意：尚未检测到环境变量 {env_name}，如需构建向量库，请先设置它。")

# VectorDatabaseBuilder 会自动读取 config/application.yaml 与 model_roles.yaml，
# 完成数据加载 → 样本生成 → 文本切片 → 构建向量库 等全部步骤。
# 用户无需关注内部细节，只需调用 invoke() 即可。

from langchain.schema import Document

def pretty_print_docs(docs, max_chars=200):
    """
    将检索结果以整洁的方式展示给用户（极其适合 quickstart）。
    """
    print(f"检索到 {len(docs)} 条文档：\n")
    for idx, doc in enumerate(docs, 1):
        content = doc.page_content.replace("\n", " ").strip()
        snippet = content[:max_chars] + ("..." if len(content) > max_chars else "")
        print(f"[{idx}] {snippet}\n")



In [ ]:
from rag_eval import VectorDatabaseBuilder,RagRunner
vector_manager = VectorDatabaseBuilder().invoke() # 建立数据库、返回数据库管理器

In [ ]:
question = "《战国无双3》是由哪两个公司合作开发的？"
docs = vector_manager.invoke(question)
pretty_print_docs(docs)

In [ ]:
runner = RagRunner(vector_manager,rag_type = "normal_rag")
result = runner.invoke(question)
result


In [ ]:
from rag_eval import EvalEngine

eval_result = EvalEngine().invoke(runner)  # limit 由 evaluation.sample_limit 控制，0 或未配置则评估全部
eval_result.show_console(top_n=5)